In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [6]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [7]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")

agent = create_agent(
    model=model,
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [9]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

I don't have direct access to your database or its structure. To determine the number of artists, you would need to:

1. Check your database schema (e.g., a table like `artists` with a `count` column).
2. Use a SQL query like `SELECT COUNT(*) FROM artists;` (if applicable).
3. Consult documentation or provide specific details about your database.

If you share more context, I can guide you further!
